In [31]:
from warnings import filterwarnings
filterwarnings('ignore')

In [32]:
import pandas as pd
df = pd.read_csv('con_dataset.csv')

In [33]:
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,Yes,162.0,2.5,1040.0,676.0,28.0,79.99
1,540.0,0.0,Yes,162.0,2.5,1055.0,676.0,28.0,61.89
2,332.5,142.5,Yes,228.0,0.0,NaN,594.0,270.0,40.27
3,332.5,142.5,No,228.0,0.0,932.0,594.0,365.0,41.05
4,198.6,132.4,No,192.0,0.0,978.4,825.5,360.0,44.30


In [34]:
df.isna().sum()

Cement                4
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      4
Fine Aggregate        0
Age                   4
Strength              0
dtype: int64

# Seperat the X and Y

In [35]:
X= df.drop(columns=['Strength'])

In [36]:
Y = df[['Strength']]

In [37]:
Y

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30
...,...
1025,44.28
1026,31.18
1027,23.70
1028,32.77


In [38]:
df.dtypes

Cement                float64
Blast Furnace Slag    float64
Fly Ash                object
Water                 float64
Superplasticizer      float64
Coarse Aggregate      float64
Fine Aggregate        float64
Age                   float64
Strength              float64
dtype: object

In [39]:
df.columns

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [40]:
cat=list(X.columns[X.dtypes == 'object'])

In [41]:
con = list(X.columns[X.dtypes != 'object'])

In [42]:
cat 

['Fly Ash']

In [43]:
con

['Cement',
 'Blast Furnace Slag',
 'Water',
 'Superplasticizer',
 'Coarse Aggregate',
 'Fine Aggregate',
 'Age']

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # Fill the Missing Value
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder 
# Scalar == Scaling Two Type 1) Standard -- X - Xm / SD  --- Range -3 tp 3   2) Normalition -- X - Xmin / Xmax - Xmin  Range - -1 to 1
# OneHot Encoder = Convert Categorical Features into Continous --- Create Multiple Columns -- Each Wrod One Columns e.g Apple --- 
# new columns will get created for Apple ---- where ever there is apple entry it will show 1 otherwise 0 
# Ordinal Encoder - used for Ordinal data like S M L XL , Only One Columns will be get created it will have value like S-1, M-2, L-3, XL-4
from sklearn.compose import ColumnTransformer # Joining the Cat and Con Columns 

In [45]:
#Create Numerical Pipeline 
num_pipe = Pipeline(steps= [("imputer", SimpleImputer(strategy='mean')),('Scalar', StandardScaler()) ] ) 

# Create Categorical Pipeline 
cat_Pipe = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                           ('ordinal', OrdinalEncoder()),
                           ('scalar', StandardScaler())
                           ])

#Combine categorical and continuous pipelines
pre = ColumnTransformer([('num', num_pipe, con), ('cat', cat_Pipe, cat)])

In [46]:
# Transform the Array 
X_pre = pre.fit_transform(X)
X_pre

array([[ 2.49569686, -0.85688789, -0.91676439, ..., -1.21767004,
        -0.27647427,  0.94887776],
       [ 2.49569686, -0.85688789, -0.91676439, ..., -1.21767004,
        -0.27647427,  0.94887776],
       [ 0.49970827,  0.79552649,  2.17546125, ..., -2.24091709,
         3.6148215 ,  0.94887776],
       ...,
       [-1.27022859,  0.75957923,  0.52158905, ...,  0.0801067 ,
        -0.27647427, -1.05387653],
       [-1.16826484,  1.30806485, -0.2795785 , ...,  0.19116644,
        -0.27647427,  0.94887776],
       [-0.18902803,  0.30849909,  0.89171909, ..., -0.15074782,
        -0.27647427,  0.94887776]])

In [47]:
# Get the features / Columns names 
cols = pre.get_feature_names_out()
cols

array(['num__Cement', 'num__Blast Furnace Slag', 'num__Water',
       'num__Superplasticizer', 'num__Coarse Aggregate',
       'num__Fine Aggregate', 'num__Age', 'cat__Fly Ash'], dtype=object)

In [48]:
# Convert into Data Frame 
X_pre = pd.DataFrame(X_pre, columns=cols)
X_pre

,num__Cement,num__Blast Furnace Slag,num__Water,num__Superplasticizer,num__Coarse Aggregate,num__Fine Aggregate,num__Age,cat__Fly Ash
0,2.495697,-0.856888,-0.916764,-0.620448,0.862035,-1.217670,-0.276474,0.948878
1,2.495697,-0.856888,-0.916764,-0.620448,1.055123,-1.217670,-0.276474,0.948878
2,0.499708,0.795526,2.175461,-1.039143,0.000000,-2.240917,3.614821,0.948878
3,0.499708,0.795526,2.175461,-1.039143,-0.528203,-2.240917,5.142396,-1.053877
4,-0.788306,0.678408,0.488793,-1.039143,0.069084,0.647884,5.061998,-1.053877
...,...,...,...,...,...,...,...,...
1025,-0.039930,0.488235,-0.092171,0.451410,-1.325016,-0.065893,-0.276474,0.948878
1026,0.400630,-0.856888,0.676200,0.702626,-1.996964,0.496893,-0.276474,0.948878
1027,-1.270229,0.759579,0.521589,-0.017528,-1.037957,0.080107,-0.276474,-1.053877
1028,-1.168265,1.308065,-0.279579,0.853356,0.213257,0.191166,-0.276474,0.948878


In [49]:
## Feature Selection
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

In [50]:
lr = LinearRegression()
sel = SequentialFeatureSelector(lr, direction='forward', n_jobs=-1)
sel_features = sel.fit_transform(X_pre, Y)


In [51]:
imp_cols = sel.get_feature_names_out()
imp_cols

array(['num__Cement', 'num__Blast Furnace Slag', 'num__Superplasticizer',
       'num__Age'], dtype=object)

In [53]:
imp_cols[0].split("__")[1]

'Cement'

In [57]:
sel_cols = []
for i in imp_cols:
    s = i.split("__")[1]
    sel_cols.append(s)